Importar las librerias

In [41]:
#Pytorch
import torch
import torch.nn as nn
import torch.optim as optim
#Sirve para transformar la imagen en bytes 
from torchvision import datasets, transforms

In [3]:
# con esto vemos si tenemos disponible una placa de video para procesar, sino utiliza el cpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Dispositivo de cómputo:", device)

Dispositivo de cómputo: cpu


In [4]:
# Definir transformaciones de las imagenes
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

In [13]:
# Cargar datos de entrenamiento y prueba
train_data = datasets.ImageFolder(root=".\\train", transform=transform)
#con esto se generan lotes chiquitos para el entrenamiento 
trainloader = torch.utils.data.DataLoader(train_data, batch_size=4, shuffle=True)

test_data = datasets.ImageFolder(root=".\\test", transform=transform)
testloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False)

In [19]:
# Definir la arquitectura de la red neuronal
class MiRedNeuronal(nn.Module):
    def __init__(self):
        super(MiRedNeuronal, self).__init__()
        self.fc1 = nn.Linear(48 * 48, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64, 4)

    def forward(self, x):
        x = x.view(-1, 48 * 48)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [20]:
model = MiRedNeuronal().to(device)

In [21]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [22]:
# Entrenamiento del modelo
num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 1000 == 999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

print('Entrenamiento finalizado')

[1,  1000] loss: 1.349
[1,  2000] loss: 1.339
[2,  1000] loss: 1.305
[2,  2000] loss: 1.299
[3,  1000] loss: 1.275
[3,  2000] loss: 1.273
[4,  1000] loss: 1.258
[4,  2000] loss: 1.246
[5,  1000] loss: 1.242
[5,  2000] loss: 1.233
[6,  1000] loss: 1.225
[6,  2000] loss: 1.221
[7,  1000] loss: 1.217
[7,  2000] loss: 1.210
[8,  1000] loss: 1.202
[8,  2000] loss: 1.202
[9,  1000] loss: 1.187
[9,  2000] loss: 1.192
[10,  1000] loss: 1.189
[10,  2000] loss: 1.180
Entrenamiento finalizado


In [23]:
# Evaluación del modelo
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Precisión del modelo en el conjunto de datos de prueba: %d %%' % (
    100 * correct / total))

Precisión del modelo en el conjunto de datos de prueba: 46 %


In [40]:
from PIL import Image
# Cargar la imagen de entrada
image_path = ".//train/happy/Training_99599521.jpg"
input_image = Image.open(image_path)
input_tensor = transform(input_image).unsqueeze(0)  # Añadir una dimensión adicional para el mini lote

# Mover la imagen al dispositivo de cómputo (CPU o GPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input_tensor = input_tensor.to(device)

# Pasar la imagen a través del modelo para obtener las predicciones
with torch.no_grad():
    model.eval()  # Cambiar al modo de evaluación
    outputs = model(input_tensor)

# Obtener la emoción predicha
_, predicted = torch.max(outputs, 1)
emotion_labels = ['enojo','miedo','felicidad','tristeza']
predicted_emotion = emotion_labels[predicted.item()]

print(predicted_emotion)



miedo
